In [ ]:
import numpy as np
from multiprocessing import Pool

In [ ]:
#!mkdir clean_files
#!mkdir html_files

In [ ]:
import pickle
from bs4 import BeautifulSoup


def make_marks(content):
    # Удаляем вставленные в текст блоки кода, оставляем пометки
    for codeBlock in content.find_all('code'):
        codeBlock.string = '<deleted block of code mark>'
    # Вместо картинок тоже оставляем пометки, т.к. без них текст часто теряет смысл
    for image in content.find_all('img'):
        image.string = '<deleted image mark>'
    for heading in content.find_all('h3'):
        heading.string = '<header start mark>' + heading.string + '<header end mark>'
    for url in content.find_all('a'):
        try:
            url.string = '<in text url start mark: ' + url['href'] + '>' 
            + url.string + '<in text url end mark>'
        except:
            pass
    return content 


def parse(page):
    soup = BeautifulSoup(page, 'html5lib') # instead of html.parser
    doc = {}
    if not soup.find("span", {"class": "post__title-text"}):
        # такое бывает, если статья не существовала или удалена
        doc['status'] = 'title_not_found'
    else:
        content = make_marks(soup.find("div", {"class": "post__text"}))

        doc['status'] = 'ok'
        doc['title'] = soup.find("span", {"class": "post__title-text"}).text
        doc['text'] = content.text
        doc['time'] = soup.find("span", {"class": "post__time"}).text
        doc['hubs'] = soup.find("ul", {"class": "post__hubs"}).text
        doc['likes'] = soup.find("span", {"class": "voting-wjt__counter"}).text
    
    return doc

In [ ]:
import requests
from bs4 import BeautifulSoup

def download_document(pid):
    """ Download and process a Habr document and its comments """
    # выгрузка документа
    r = requests.get('https://habrahabr.ru/post/' +str(pid) + '/')
    # парсинг документа
    soup = BeautifulSoup(r.text, 'html5lib') # instead of html.parser

    doc = parse(soup)
    doc['id'] = pid
    
    if 'hubs' in doc:    
        # сохранение результата в отдельный файл
        clean_filename = r'clean_files/' + str(pid) + '.pkl'
        with open(clean_filename, 'wb') as f:
            pickle.dump(doc, f)
        
        html_filename = r'html_files/' + str(pid) + '.html'
        with open(html_filename, 'w') as html_out:
            html_out.write(r.text)

In [ ]:
with Pool(10) as p:
    docs = p.imap_unordered(download_document, np.arange(150000, 545000))